<a href="https://colab.research.google.com/github/bjelikj/DT2119/blob/master/Labs/Lab3/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import numpy as np
import tensorflow as tf
import os

In [6]:
"""
Necessary files:
- lab3_proto.py
- lab3_tools.py
- lab1_proto.py
- lab1_tools.py
- prondict.py

If not uploaded to Google Drive:
- lab2_models_all.npz
- lab3_example.npz
"""
"""
if IN_COLAB:
    !rm lab1_proto.py
    !rm lab1_tools.py
    !rm lab2_proto.py
    !rm lab2_tools.py
    !rm lab3_proto.py
    !rm lab3_tools.py
    !rm lab3_example.npz
    !rm lab2_models_all.npz
    !ls
"""
from google.colab import files
files.upload()

Saving lab1_proto.py to lab1_proto.py
Saving lab1_tools.py to lab1_tools.py
Saving lab2_proto.py to lab2_proto.py
Saving lab2_tools.py to lab2_tools.py
Saving lab3_proto.py to lab3_proto.py
Saving lab3_tools.py to lab3_tools.py
Saving prondict.py to prondict.py


{'lab1_proto.py': b'# DT2119, Lab 1 Feature Extraction\r\nimport numpy as np\r\nimport matplotlib.pyplot as plt\r\nimport scipy.signal as ssi\r\nfrom scipy import fftpack\r\nfrom lab1_tools import trfbank\r\nfrom scipy.fftpack.realtransforms import dct\r\nfrom lab1_tools import *\r\nfrom scipy.spatial.distance import euclidean\r\nfrom scipy.spatial.distance import cdist\r\n\r\n# Function given by the exercise ----------------------------------\r\n\r\ndef mspec(samples, winlen=400, winshift=200, preempcoeff=0.97, nfft=512, samplingrate=20000):\r\n    """Computes Mel Filterbank features.\r\n\r\n    Args:\r\n        samples: array of speech samples with shape (N,)\r\n        winlen: lenght of the analysis window\r\n        winshift: number of samples to shift the analysis window at every time step\r\n        preempcoeff: pre-emphasis coefficient\r\n        nfft: length of the Fast Fourier Transform (power of 2, >= winlen)\r\n        samplingrate: sampling rate of the original signal\r\n\r

In [0]:
from lab1_proto import *
from lab2_proto import *
from lab3_proto import *
from lab1_tools import *
from lab2_tools import *
from lab3_tools import *
from prondict import prondict



## Check examples


In [0]:
example = np.load('/content/drive/My Drive/Lab3/lab3_example.npz', allow_pickle=True)['example'].item()

# 4.1 Target Class Definition

In [10]:
phoneHMMs = np.load('/content/drive/My Drive/Lab3/lab2_models_all.npz', allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]
print(stateList)

['ah_0', 'ah_1', 'ah_2', 'ao_0', 'ao_1', 'ao_2', 'ay_0', 'ay_1', 'ay_2', 'eh_0', 'eh_1', 'eh_2', 'ey_0', 'ey_1', 'ey_2', 'f_0', 'f_1', 'f_2', 'ih_0', 'ih_1', 'ih_2', 'iy_0', 'iy_1', 'iy_2', 'k_0', 'k_1', 'k_2', 'n_0', 'n_1', 'n_2', 'ow_0', 'ow_1', 'ow_2', 'r_0', 'r_1', 'r_2', 's_0', 's_1', 's_2', 'sil_0', 'sil_1', 'sil_2', 'sp_0', 't_0', 't_1', 't_2', 'th_0', 'th_1', 'th_2', 'uw_0', 'uw_1', 'uw_2', 'v_0', 'v_1', 'v_2', 'w_0', 'w_1', 'w_2', 'z_0', 'z_1', 'z_2']


# 4.2 Forced Alignment

In [11]:
filename = '/content/drive/My Drive/tidigits/disc_4.1.1/tidigits/train/man/nw/z43a.wav'
samples, samplingrate = loadAudio(filename)
lmfcc = mfcc(samples)
print("LMFCC Shape: ", lmfcc.shape)

wordTrans = list(path2info(filename)[2])
print("Sequence of digits: ", wordTrans)

print("Pronunciation dictionary: ", prondict)

phoneTrans = words2phones(wordTrans, prondict)
print("Phone level transcription: ", phoneTrans)
utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)

stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans
              for stateid in range(nstates[phone])]

LMFCC Shape:  (178, 13)
Sequence of digits:  ['z', '4', '3']
Pronunciation dictionary:  {'o': ['ow'], 'z': ['z', 'iy', 'r', 'ow'], '1': ['w', 'ah', 'n'], '2': ['t', 'uw'], '3': ['th', 'r', 'iy'], '4': ['f', 'ao', 'r'], '5': ['f', 'ay', 'v'], '6': ['s', 'ih', 'k', 's'], '7': ['s', 'eh', 'v', 'ah', 'n'], '8': ['ey', 't'], '9': ['n', 'ay', 'n']}
Phone level transcription:  ['sil', 'z', 'iy', 'r', 'ow', 'sp', 'f', 'ao', 'r', 'sp', 'th', 'r', 'iy', 'sp', 'sil']


In [12]:
# Compare with examples
for i in example['utteranceHMM'].keys():
    print(compare(utteranceHMM[i], example['utteranceHMM'][i]))

obsloglik = log_multivariate_normal_density_diag(lmfcc, utteranceHMM['means'], utteranceHMM['covars'])
viterbi_loglik, viterbi_path = viterbi(obsloglik, np.log(utteranceHMM['startprob']), np.log(utteranceHMM['transmat']))
viterbiStateTrans = [stateTrans[idx] for idx in viterbi_path.astype(np.int64)]

print('Comparing obsloglik...', compare(obsloglik, example['obsloglik']))
print('Comparing viterbiPath...', compare(viterbi_path, example['viterbiPath']))
print('Comparing viterbiStateTrans...', np.all(viterbiStateTrans == example['viterbiStateTrans']))

frames = frames2trans(viterbiStateTrans, outfilename= ''.join(path2info(filename)[2:]) + '.lab')

True
True
True
True
Comparing obsloglik... True
Comparing viterbiPath... True
Comparing viterbiStateTrans... True


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [0]:
#from IPython.display import Image
#Image(filename='/content/drive/My Drive/z43a_wavesurfer.png')

# 4.3 Feature Extraction

In [0]:
def extract_features_and_targets(filename):
    """ extract_features_and_targets: extracts lmfcc, mspecc and 
        targets from a *.wav sound file

    Args:
        filename: location of .wav file

    Returns:
        lmfcc: liftered mfcc features
        mspec: filterbank features
        targets: indices of the target state for each feature
    """
    samples, samplingrate = loadAudio(filename)
    wordTrans = list(path2info(filename)[2])
    phoneTrans = words2phones(wordTrans, prondict)
    stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans 
                  for stateid in range(nstates[phone])]
    utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)

    stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans
              for stateid in range(nstates[phone])]

    lmfcc = mfcc(samples)
    mspec_ = mspec(samples)

    obsloglik = log_multivariate_normal_density_diag(lmfcc, utteranceHMM['means'], utteranceHMM['covars'])
    viterbi_loglik, viterbi_path = viterbi(obsloglik, np.log(utteranceHMM['startprob']), np.log(utteranceHMM['transmat']), True)
    targets = [stateTrans[idx] for idx in viterbi_path.astype(np.int64)] 

    return lmfcc, mspec_, targets

Extract and save lmfcc and mspec features, and their associated targets

In [15]:
"""
from tqdm import tqdm
traindata = []
for root, dirs, files in tqdm(os.walk('/content/drive/My Drive/tidigits/disc_4.1.1/tidigits/train'), desc='dirs'):
  for file in files:
    if file.endswith('.wav'):
      filename = os.path.join(root, file)
      samples, samplingrate = loadAudio(filename)
      lmfcc, mspec_, targets =  extract_features_and_targets(filename)
      traindata.append({'filename': filename, 'lmfcc': lmfcc,
                        'mspec': mspec_, 'targets': targets})

np.savez('/content/drive/My Drive/Lab3/traindata.npz', traindata=traindata)
"""

"\nfrom tqdm import tqdm\ntraindata = []\nfor root, dirs, files in tqdm(os.walk('/content/drive/My Drive/tidigits/disc_4.1.1/tidigits/train'), desc='dirs'):\n  for file in files:\n    if file.endswith('.wav'):\n      filename = os.path.join(root, file)\n      samples, samplingrate = loadAudio(filename)\n      lmfcc, mspec_, targets =  extract_features_and_targets(filename)\n      traindata.append({'filename': filename, 'lmfcc': lmfcc,\n                        'mspec': mspec_, 'targets': targets})\n\nnp.savez('/content/drive/My Drive/Lab3/traindata.npz', traindata=traindata)\n"

In [16]:
"""
testdata = []
for root, dirs, files in tqdm(os.walk('/content/drive/My Drive/tidigits/disc_4.2.1/tidigits/test'), desc='dirs'):
  for file in files:
    if file.endswith('.wav'):
      filename = os.path.join(root, file)
      samples, samplingrate = loadAudio(filename)
      lmfcc, mspec_, targets =  extract_features_and_targets(filename)
      testdata.append({'filename': filename, 'lmfcc': lmfcc,
                        'mspec': mspec_, 'targets': targets})

np.savez('/content/drive/My Drive/Lab3/testdata.npz', testdata=testdata)
"""

"\ntestdata = []\nfor root, dirs, files in tqdm(os.walk('/content/drive/My Drive/tidigits/disc_4.2.1/tidigits/test'), desc='dirs'):\n  for file in files:\n    if file.endswith('.wav'):\n      filename = os.path.join(root, file)\n      samples, samplingrate = loadAudio(filename)\n      lmfcc, mspec_, targets =  extract_features_and_targets(filename)\n      testdata.append({'filename': filename, 'lmfcc': lmfcc,\n                        'mspec': mspec_, 'targets': targets})\n\nnp.savez('/content/drive/My Drive/Lab3/testdata.npz', testdata=testdata)\n"

# 4.4 Training and Validation Sets

##### Split the training data into a training set (roughly 90%) and validation set (remaining 10%). Make sure that there is a similar distribution of men and women in both sets, and that each speaker is only included in one of the two sets. The last requirement is to ensure that we do not get artificially good results on the validation set. Explain how you selected the two data sets.

*** You should specify both the training and validation data with the respective targets. What is the purpose of the validation data? **

In [0]:
train_data = np.load('/content/drive/My Drive/Lab3/traindata.npz', allow_pickle=True)['traindata']

In [0]:
def train_val_split(data):
  """train_val_split: splits dataset into train and validation sets

  Args:
    data: dataset to be split
  """
  MAN_SAMPLES = 4235
  WOMAN_SAMPLES = 4388
  VAL_PER = 0.1
  SAMPLES_PER_PERSON = 77

  nsamples = len(data)
  approx_val_size = int(0.1 * nsamples)
  val_class_people_count = int(approx_val_size / (SAMPLES_PER_PERSON))

  if val_class_people_count % 2 != 0:
      val_class_people_count += 1

  val_size = val_class_people_count * SAMPLES_PER_PERSON
  training_size = nsamples - val_size

  samples_per_gender = int(val_size / 2)

  val_data = [data[i] for i in range(0, samples_per_gender)]
  val_data.extend([data[i] for i in range(MAN_SAMPLES, MAN_SAMPLES + samples_per_gender)])
  training_data = [sample for sample in data if sample['filename'] not in [x['filename'] for x in val_data]]

  np.save('/content/drive/My Drive/Lab3/training_data.npy', training_data)
  np.save('/content/drive/My Drive/Lab3/validation_data.npy', val_data)

In [0]:
train_val_split(train_data)

# 4.5 Acoustic Context (Dynamic Features)

In [0]:
train_data = np.load('/content/drive/My Drive/Lab3/training_data.npy', allow_pickle=True)
val_data = np.load('/content/drive/My Drive/Lab3/validation_data.npy', allow_pickle=True)
test_data = np.load('/content/drive/My Drive/Lab3/testdata.npz', allow_pickle=True)['testdata']

In [0]:
# For each utterance and time step, stack 7 MFCC or filterbank features symmetrically distributed
# around the current time step. That is, at time n, stack the features at times [n-3; n-
# 2; n-1; n; n+1; n+2; n+3]). At the beginning and end of each utterance, use mirrored feature
# vectors in place of the missing vectors. For example at the beginning use feature vectors with
# indexes [3; 2; 1; 0; 1; 2; 3] for the first time step, [2; 1; 0; 1; 2; 3; 4] for the second time step, and so
# on. The “boundary effect” is usually not very important because each utterance begins and ends
# with silence.

In [0]:
from tqdm import tqdm
def create_dynamic_features(data):
    """ create_dynamic_features: Creates dynamic features by concatinated 7 
        lmfcc or mspec featues together for each step

    Args:
        data: list of dictionaries with keys: 'lmfcc', 'mspec' and 'targets'
    
    Returns:
        lmfcc_features: [NxD_lmfcc * 7] where N = number of all concatinated samples of all words in data
        mspec_features: [NxD_mspec * 7] where N = of all concatinated samples of all words in data
        targets: [N,] index of state for each sample
    """
    
    D_lmfcc = data[0]['lmfcc'].shape[1]
    D_mspec = data[0]['mspec'].shape[1]
    N = sum([len(x['targets']) for x in data])

    # Features to be returned
    lmfcc_features = np.zeros((N, D_lmfcc * 7))
    mspec_features = np.zeros((N, D_mspec * 7))
    
    # Targets to be returned
    targets = []

    # through all data
    k = 0
    for x in tqdm(data): 
        times, dim = x['lmfcc'].shape
        # for each time step
        for i in range(times):
            if i < 3 or i >= times - 3:
                lmfcc_features[k, :] = np.hstack(np.pad(x['lmfcc'], pad_width=((3, 3), (0, 0)), mode='reflect')[i:i+7, :])
                mspec_features[k, :] = np.hstack(np.pad(x['mspec'], pad_width=((3, 3), (0, 0)), mode='reflect')[i:i+7, :])
            else:
                lmfcc_features[k,:] = np.hstack(x['lmfcc'][i-3:i+4, :])
                mspec_features[k,:] = np.hstack(x['mspec'][i-3:i+4, :])
            k +=1
        targets = targets + x['targets']

    return lmfcc_features, mspec_features, targets

In [23]:
lmfcc_train_x, mspec_train_x, train_y = create_dynamic_features(train_data)
lmfcc_val_x, mspec_val_x, val_y = create_dynamic_features(val_data)
lmfcc_test_x, mspec_test_x, test_y = create_dynamic_features(test_data)

100%|██████████| 8700/8700 [01:18<00:00, 111.40it/s]


# 4.6 Feature Standardisation

Normalization over the complete training set

In [24]:
from sklearn.preprocessing import StandardScaler
"""
print('Train data before feature standardization: ', lmfcc_train_x[0])
print('Validation data before feature standardization: ', lmfcc_val_x[0])
print('Test data before feature standardization: ', lmfcc_test_x[0])
"""
scaler = StandardScaler()
scaler = scaler.fit(lmfcc_train_x)
lmfcc_train_x = scaler.transform(lmfcc_train_x)
lmfcc_val_x = scaler.transform(lmfcc_val_x)
lmfcc_test_x = scaler.transform(lmfcc_val_x)
"""
print('Train data after feature standardization: ', lmfcc_train_x[0])
print('Validation data after feature standardization: ', lmfcc_val_x[0])
print('Test data after feature standardization: ', lmfcc_test_x[0])
"""

Train data before feature standardization:  [ 4.45740695e+01 -9.70452188e+01  9.89285496e+01  1.30110535e+01
 -2.22716868e+01 -8.93713732e+01 -1.93372222e+01 -2.46926385e+02
 -2.42524656e-01  1.21487772e+02 -1.17587658e+01  1.40877864e+01
 -1.64728422e+01  2.03107423e+01 -1.58162304e+02  1.29291883e+02
  6.78660270e+01  8.23696302e+01  3.94972794e+01 -9.39323913e+01
 -2.08224406e+02 -7.76468906e+01  2.08048811e+02 -9.59758683e+01
 -1.62394674e+02  9.61689525e+01  3.30028013e+01 -1.44811196e+02
  1.31217981e+02  1.65058619e+01  6.52676127e+01  3.08077257e+01
 -1.77337209e+02 -1.81096519e+02 -5.86573894e+01 -3.31812615e+01
 -1.27537435e+01  3.48484909e+01  1.08079158e+02  2.25167437e+01
 -1.41375676e+02  1.01894601e+02  7.60806414e+01 -3.51136995e+01
 -1.03045435e+02 -6.94672047e+00 -1.14077622e+02 -1.43544505e+02
 -5.66687693e+01 -7.28719949e+00 -3.18410929e+01  6.32642360e+01
  3.30028013e+01 -1.44811196e+02  1.31217981e+02  1.65058619e+01
  6.52676127e+01  3.08077257e+01 -1.77337209e+

Conversion of features to 'float32' type, and target arrays to Keras categorical format

In [25]:
from keras.utils import np_utils

lmfcc_train_x = lmfcc_train_x.astype('float32')
lmfcc_val_x = lmfcc_val_x.astype('float32')
lmfcc_test_x = lmfcc_test_x.astype('float32')

# train_y_test = list.copy(train_y)
# for idx, state in enumerate(train_y):
#   train_y_test[idx] = stateList.index(state)

list_targets = [train_y, val_y, test_y]

for dataset in list_targets:
  for idx, state in enumerate(dataset):
    dataset[idx] = stateList.index(state)

train_y = np_utils.to_categorical(train_y, len(stateList))
val_y = np_utils.to_categorical(val_y, len(stateList))
test_y = np_utils.to_categorical(test_y, len(stateList))

Using TensorFlow backend.


In [26]:
print(len(train_y))
print(train_y[0])

1343422
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#5 Phoneme Recognition with Deep Neural Networks

Here are the minimum list of configurations to test, but you can test your favourite models if you manage to run the training in reasonable time. 

1. input: liftered MFCCs, one to four hidden layers of size 256, rectified linear units
2. input: filterbank features, one to four hidden layers of size 256, rectified linear units
3. same as 1. but with dynamic features as explained in Section 4.5
4. same as 2. but with dynamic features as explained in Section 4.5

Note the evolution of the loss function and the accuracy of the model for every epoch. What can you say comparing the results on the training and validation data? There are many other parameters that you can vary, if you have time to play with the models.
For example:

• different activation functions than ReLU

• different number of hidden layers

• different number of nodes per layer

• different length of context input window

• strategy to update learning rate and momentum

• initialisation with DBNs instead of random

• different normalisation of the feature vectors

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

## lmfcc features

In [31]:
lmfcc_train_x.shape

(1343422, 91)

In [36]:
model = Sequential()

# A typical value for hidden nodes is 256, but you can experiment with this to see if convergence becomes faster or
slower.
model.add(Dense(64, input_shape=(91,), activation='relu', dtype="float32"))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(61))
model.add(Activation('softmax', name='posterior'))

opt = Adam()
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 64)                5888      
_________________________________________________________________
dense_13 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_14 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_15 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 61)                3965      
_________________________________________________________________
posterior (Activation)       (None, 61)               

In [37]:
filepath = "/content/drive/My Drive/Lab3/weights.{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')

model.fit(lmfcc_train_x, train_y, epochs=10, batch_size=64,
          validation_data = (lmfcc_val_x, val_y), callbacks=checkpoint)

Epoch 1/10
20991/20991 [==============================] - 62s 3ms/step - loss: 0.8520 - accuracy: 0.7214 - val_loss: 0.7253 - val_accuracy: 0.7632
Epoch 2/10
20991/20991 [==============================] - 62s 3ms/step - loss: 0.7031 - accuracy: 0.7652 - val_loss: 0.7050 - val_accuracy: 0.7687
Epoch 3/10
20991/20991 [==============================] - 61s 3ms/step - loss: 0.6722 - accuracy: 0.7748 - val_loss: 0.7142 - val_accuracy: 0.7668
Epoch 4/10
20991/20991 [==============================] - 63s 3ms/step - loss: 0.6543 - accuracy: 0.7805 - val_loss: 0.6984 - val_accuracy: 0.7721
Epoch 5/10
20991/20991 [==============================] - 61s 3ms/step - loss: 0.6426 - accuracy: 0.7839 - val_loss: 0.6914 - val_accuracy: 0.7747
Epoch 6/10
20991/20991 [==============================] - 61s 3ms/step - loss: 0.6338 - accuracy: 0.7864 - val_loss: 0.6950 - val_accuracy: 0.7770
Epoch 7/10
20991/20991 [==============================] - 62s 3ms/step - loss: 0.6273 - accuracy: 0.7883 - val_loss: 0

In [41]:
print("Evaluating model on test data...")
results = model.evaluate(lmfcc_test_x, test_y)
print("Test loss:", results[0])
print("Test acc:", results[1])

# IT NEEDS THE NON ONE HOT ENCODED HERE, WE SHOULD SAVE A COPY
"""
y_test_pred = np.argmax(model.predict(x_test), axis=1)

print("Confusion matrix:")
print(confusion_matrix(y_test, y_test_pred))
print("Classification reprot:")
print(classification_report(y_test, y_test_pred, digits=8))
"""

Evaluating model on test data...


ValueError: ignored

In [43]:
print(lmfcc_test_x.shape)
print(test_y.shape)

(163635, 91)
(1526682, 61)
